<a href="https://colab.research.google.com/github/DineshY1011/US_Accident/blob/main/Milestone_3/Week_6/Day_28/Cat_Boost_GBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### CatBoost GBM Classifier Model Training

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 4.9 MB/s eta 0:00:00


In [4]:
# Load necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.impute import SimpleImputer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier

In [5]:
df = pd.read_csv("/content/drive/MyDrive/US_Dataset/Cleaned_1M_Dataset/cleaned_us_accidents.csv")
# df = df.sample(100000, random_state=42)


target = 'Severity'
X = df.drop(columns=target)
y = df[target]

In [6]:
categorical_cols = X.select_dtypes(include='object').columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64', 'bool']).columns.tolist()

In [7]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [9]:
clf_catboost = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', CatBoostClassifier(
        iterations=300,
        learning_rate=0.05,
        depth=8,
        loss_function='MultiClass',
        eval_metric='Accuracy',
        random_seed=42,
        verbose=100,
        early_stopping_rounds=30
    ))
])

clf_catboost.fit(X_train, y_train)

0:	learn: 0.8731024	total: 5.28s	remaining: 26m 19s
100:	learn: 0.8804987	total: 9m 29s	remaining: 18m 42s
200:	learn: 0.8838437	total: 19m 18s	remaining: 9m 30s
299:	learn: 0.8848603	total: 29m 29s	remaining: 0us


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Start_Lat', 'Start_Lng',
                                                   'Distance(mi)',
                                                   'Temperature(F)',
                                                   'Wind_Chill(F)',
                                                   'Humidity(%)',
                                                   'Pressure(in)',
                                                   'Visibility(mi)',
                                                   'Wind_Speed(mph)',
                                                   'Precipitation(in)',
                                                   'Amenity', 'Bump',
                                                   'Crossing', 'Gi...
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Start_Time', 'End_Time',
                                                   'City', 'County', 'State',
                                                   'Zipcode', 'Wind_Direction',
                                                   'Weather_Condition',
                                                   'Sunrise_Sunset',
                                                   'Civil_Twilight'])])),
                ('classifier',
                 <catboost.core.CatBoostClassifier object at 0x7e3bb9301490>)])

In [10]:
y_pred_cb = clf_catboost.predict(X_test)

print('CatBoost Classifier Accuracy:', accuracy_score(y_test, y_pred_cb))
print('\nClassification Report:\n', classification_report(y_test, y_pred_cb))
print('\nConfusion Matrix:\n', confusion_matrix(y_test, y_pred_cb))

CatBoost Classifier Accuracy: 0.8838925272683722

Classification Report:
               precision    recall  f1-score   support

           1       0.00      0.00      0.00       166
           2       0.89      0.99      0.94    149447
           3       0.62      0.27      0.37     13959
           4       0.62      0.02      0.03      7597

    accuracy                           0.88    171169
   macro avg       0.53      0.32      0.33    171169
weighted avg       0.86      0.88      0.85    171169


Confusion Matrix:
 [[     0    165      1      0]
 [     0 147471   1943     33]
 [     0  10213   3706     40]
 [     0   7116    363    118]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
